In [ ]:
import pandas as pd
import re
from sklearn.metrics import cohen_kappa_score
from difflib import SequenceMatcher

# === 1. Загрузка данных ===
df1 = pd.read_excel("/content/М58 говорит правду_1.xlsx")      # транскрипт аннотатора 1
df2 = pd.read_excel("/content/М58 говорит правду_2.xlsx")  # транскрипт аннотатора 2

# === 2. Извлекаем нужные столбцы ===
segs1 = df1["ЭДЕ"].dropna().astype(str).tolist()
segs2 = df2["ЭДЕ"].dropna().astype(str).tolist()

# === 3. Нормализация текста ===
def normalize_text(s):
    s = s.lower()
    s = re.sub(r"\{[^}]*\}", " ", s)   # удалить {СМЕХ}, {ВЗДОХ} и др.
    s = re.sub(r"\([^)]*\)", " ", s)   # удалить (0.4) и подобное
    s = re.sub(r"\|[A-Z]\|", " ", s)   # удалить служебные |E|, |S|
    s = re.sub(r"\*[^*]*\*", " ", s)   # удалить *Смеётся* и подобное
    s = re.sub(r"[^\w\sа-яё-]", " ", s) # оставить буквы, цифры, дефис
    s = re.sub(r"\s+", " ", s).strip()
    return s

text1 = normalize_text(" ".join(segs1))
text2 = normalize_text(" ".join(segs2))

# Добавляем вывод нормализованного текста
print("Нормализованный текст (аннотатор 1):")
print(text1)
print("\nНормализованный текст (аннотатор 2):")
print(text2)


tokens1 = text1.split()
tokens2 = text2.split()

# === 4. Обрезаем по длине меньшего текста ===
L = min(len(tokens1), len(tokens2))
tokens1 = tokens1[:L]
tokens2 = tokens2[:L]

# === 5. Бинарное совпадение слов ===
matches = [1 if tokens1[i] == tokens2[i] else 0 for i in range(L)]
annotator1 = [1]*L  # "эталон" ряд: 1 = слово есть
annotator2 = matches

# === 6. Каппа Коэна ===
kappa = cohen_kappa_score(annotator1, annotator2)

# === 7. Дополнительные метрики ===
agreement_ratio = sum(matches) / L

# Функция для WER и CER
def levenshtein(a, b):
    n, m = len(a), len(b)
    if n == 0: return m
    if m == 0: return n
    prev = list(range(m+1))
    for i in range(1, n+1):
        cur = [i] + [0]*m
        for j in range(1, m+1):
            cost = 0 if a[i-1] == b[j-1] else 1
            cur[j] = min(prev[j]+1, cur[j-1]+1, prev[j-1]+cost)
        prev = cur
    return prev[m]

WER = levenshtein(tokens1, tokens2) / max(1, len(tokens1))
CER = levenshtein(list(text1), list(text2)) / max(1, len(text1))

# === 8. Вывод результатов ===
print(f"\nКоличество слов (min length): {L}")
print(f"Доля совпадений по позициям: {agreement_ratio:.3f}")
print(f"Коэффициент Каппа Коэна: {kappa:.3f}")
print(f"Word Error Rate (WER): {WER:.3f}")
print(f"Character Error Rate (CER): {CER:.3f}")

Нормализованный текст (аннотатор 1):
а я хочу ам поделиться м радостным событием моему замечательному другу и коллеге на днях исполнилась а круглая дата и я был а приглашен на ам замечательный вечер юбилей а вот где мой замечательный друг играл спектакль а в роли он был в роли алексея максимовича горького и м зовут этого замечательного человека игорь олегович смеловский во-от а мне было es а я был в курсе всего этого э дела подготовки к этому вечеру да я а м даже принимал какое-то участие так вот и когда а м случился а этот спектакль настал этот эм в-ве день а вот естественно он пригласил очень много госте-ей а вот и а труппу а м пригласил на свой юбилейный вечер а вот и-и к этому времени уже м шла какая-то подгото-овка готовилась ам какие-то блю-юда какие-то всевозможные напи-итки всевозможные а закуски и так далее и тому подобное но самое главное то что а игорь олегович а очень переживал что у у него очень много было желающих попасть на этот спектакль на этот юбилей а м как оказалось

In [ ]:
import re
import pandas as pd
from pathlib import Path
from collections import Counter
import numpy as np
from sklearn.metrics import cohen_kappa_score
#import krippendorff

# === Пути к файлам ===
FILE1 = "/content/М58 говорит правду_1.xlsx"
FILE2 = "/content/М58 говорит правду_2.xlsx"
COLNAME = "ЭДЕ"

# === Вспомогательные функции ===

def read_and_concat(filepath, col=COLNAME):
    """Читает xlsx и склеивает все непустые значения столбца col в один текст.
       После каждой строки добавляется метка [EDU].
    """
    df = pd.read_excel(filepath)
    if col not in df.columns:
        raise ValueError(f"В файле {Path(filepath).name} нет колонки '{col}'. Доступные: {df.columns.tolist()}")

    lines = df[col].dropna().astype(str).tolist()
    text_with_edu = " [EDU] ".join(lines) + " [EDU]"
    return text_with_edu

def separate_ellipsis_from_words(text):
    text = re.sub(r"(?P<char>[\wа-яёА-ЯЁ])(\.{3,})(?=\s*\()", r"\1 \2", text)
    return text

def replace_silent_pauses(text):
    silent_pattern = re.compile(r"\.{3,}\s*\(\s*\d*(?:\.\d+)?\s*\)", re.IGNORECASE)
    return silent_pattern.sub(" [silent_pause] ", text)

def replace_filled_pauses(text):
    filled_pattern = re.compile(r"\b[аеёиоуыэюяём]{1,3}\s*\(\s*[\d\.]*\s*\)", re.IGNORECASE)
    return filled_pattern.sub(" [filled_pause] ", text)

def replace_any_noise(text):
    noise_pattern = re.compile(r"\{[^}]*\}", re.IGNORECASE)
    return noise_pattern.sub(" [any_noise] ", text)

def normalize_text_preserve_markers(text):
    parts = re.split(r"(\[silent_pause\]|\[filled_pause\]|\[any_noise\]|\[EDU\])", text)
    normalized_parts = []
    for part in parts:
        if re.fullmatch(r"\[silent_pause\]|\[filled_pause\]|\[any_noise\]|\[EDU\]", part):
            normalized_parts.append(part)
        else:
            p = part.lower()
            p = p.replace("ё", "е")
            p = re.sub(r"\|[A-Za-zА-Яа-яЁё]+\|", " ", p)
            p = re.sub(r"[=|]+", " ", p)
            p = re.sub(r"\*+", " ", p)
            p = re.sub(r"<[^>]*>", " ", p)
            p = re.sub(r"\b([а-яё])(?:-+\1)+\b", r"\1", p, flags=re.IGNORECASE)
            p = re.sub(r"([а-яё])\1{2,}", r"\1", p, flags=re.IGNORECASE)
            p = re.sub(r"[^\w\s\[\]\-]", " ", p)
            p = re.sub(r"\s+", " ", p).strip()
            normalized_parts.append(p)
    normalized_text = " ".join([p for p in normalized_parts if p])
    normalized_text = re.sub(r"\s+\[", " [", normalized_text)
    normalized_text = re.sub(r"\]\s+", "] ", normalized_text)
    return normalized_text.strip()

def count_markers(text):
    """Подсчёт маркеров в тексте"""
    c = Counter()
    c["filled_pause"] = len(re.findall(r"\[filled_pause\]", text))
    c["silent_pause"] = len(re.findall(r"\[silent_pause\]", text))
    c["any_noise"] = len(re.findall(r"\[any_noise\]", text))
    c["EDU"] = len(re.findall(r"\[EDU\]", text))
    return c

def marker_vector(text, marker):
    """Создаёт бинарный вектор по маркеру"""
    words = text.split()
    vector = []
    for i, w in enumerate(words):
        if w.startswith("[") and w.endswith("]"):
            continue
        if i + 1 < len(words) and words[i + 1] == marker:
            vector.append(1)
        else:
            vector.append(0)
    return np.array(vector)

# === Основной пайплайн ===
def process_files(file1, file2, col=COLNAME):
    # 1–2. Собираем тексты
    text1 = read_and_concat(file1, col)
    text2 = read_and_concat(file2, col)

    # Отделяем многоточия от слов
    text1 = separate_ellipsis_from_words(text1)
    text2 = separate_ellipsis_from_words(text2)

    # Замены маркеров
    for func in [replace_silent_pauses, replace_filled_pauses, replace_any_noise]:
        text1 = func(text1)
        text2 = func(text2)

    # Нормализация
    norm1 = normalize_text_preserve_markers(text1)
    norm2 = normalize_text_preserve_markers(text2)

    # Подсчёт маркеров
    counts1 = count_markers(norm1)
    counts2 = count_markers(norm2)
    print("\n=== Количество маркеров в тексте 1 ===", dict(counts1))
    print("=== Количество маркеров в тексте 2 ===", dict(counts2))

    # Формируем бинарные вектора и считаем согласие
    markers = ["[filled_pause]", "[silent_pause]", "[any_noise]", "[EDU]"]
    results_vectors = {}
    for m in markers:
        vec1 = marker_vector(norm1, m)
        vec2 = marker_vector(norm2, m)
        percent = np.mean(vec1 == vec2)
        kappa = cohen_kappa_score(vec1, vec2)
        #alpha = krippendorff.alpha(reliability_data=np.array([vec1, vec2]), level_of_measurement='nominal')
        results_vectors[m] = {
            "vec1": vec1,
            "vec2": vec2,
            "percent": percent,
            "kappa": kappa,
            #"alpha": alpha,
            "len1": len(vec1),
            "len2": len(vec2)
        }
        print(f"\n=== {m} ===")
        print("Text1 vector length:", len(vec1))
        print("Text2 vector length:", len(vec2))
        print("Percent agreement:", percent)
        print("Cohen's kappa:", kappa)
        #print("Krippendorff alpha:", alpha)

    return {
        "norm_1": norm1,
        "norm_2": norm2,
        "counts_1": counts1,
        "counts_2": counts2,
        "vectors": results_vectors
    }

# === Запуск ===
if __name__ == "__main__":
    results = process_files(FILE1, FILE2)



=== Количество маркеров в тексте 1 === {'filled_pause': 104, 'silent_pause': 72, 'any_noise': 13, 'EDU': 202}
=== Количество маркеров в тексте 2 === {'filled_pause': 104, 'silent_pause': 58, 'any_noise': 13, 'EDU': 274}

=== [filled_pause] ===
Text1 vector length: 1223
Text2 vector length: 1223
Percent agreement: 0.9623875715453802
Cohen's kappa: 0.6031797534068787

=== [silent_pause] ===
Text1 vector length: 1223
Text2 vector length: 1223
Percent agreement: 0.9672935404742437
Cohen's kappa: 0.6752479454055418

=== [any_noise] ===
Text1 vector length: 1223
Text2 vector length: 1223
Percent agreement: 0.9918233851185609
Cohen's kappa: 0.37088477366255146

=== [EDU] ===
Text1 vector length: 1223
Text2 vector length: 1223
Percent agreement: 0.8454619787408013
Cohen's kappa: 0.470590660378871
